In [71]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from pandas import Series
from copy import deepcopy

# Import dataset
aurora = pd.read_csv("Aurora v2.0 data file - data.csv")


In [69]:
aurora.head()

,ID,Bot,QR,Asistencia,Gestor,Consentimiento,Genre,Edad,País actual,Actual otro,...,satisfaccion_otra_nna,recomendacion_otra_nna,Razones no acceso,No acceso otro,Último país,Último otro,Zona último,Último pais monitoreo,Información solicitada,Información enviada
0,308878469,Aurora v2.0,NaN,SI,José Herrera,NaN,Hombre,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tips de autocuidado,Salud mental niñ@s
1,308878469,NaN,NaN,NO,José Herrera,NO,Hombre,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ubicación de la ayuda,Centroamérica y México
2,308878469,Aurora v2.0,NaN,SI,Valentina Medina,SI,Hombre,12.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tips de seguridad,Para que no nos piquen:
3,308878469,Aurora v2.0,NaN,SI,Amanda Bogantes,SI,Hombre,47.0,Colombia,NaN,...,NaN,NaN,Me negaron el acceso a la ayuda,NaN,Nicaragua,NaN,Las Tablillas - Nicaragu,Nicaragua,Tips de autocuidado,Mascotas
4,308878469,QR-Feedback,Necoclí,SI,NaN,NO,Hombre,47.0,Colombia,NaN,...,NaN,NaN,Me negaron el acceso a la ayuda,NaN,Nicaragua,NaN,Las Tablillas - Nicaragu,Nicaragua,Políticas de migración,Derechos como migrantes


In [74]:
aurora["Bot"]

0            Aurora v2.0
1                    NaN
2            Aurora v2.0
3            Aurora v2.0
4            QR-Feedback
5            Aurora v2.0
6            QR-Feedback
7     Monitoreo-Feedback
8            QR-Feedback
9            Aurora v2.0
10                   NaN
11                   NaN
Name: Bot, dtype: object